In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics

%matplotlib inline

In [73]:
alpha = pd.read_csv('datasets/data_alpha.csv')

In [127]:
copy = alpha.iloc[::-1].copy()
copy.reset_index(drop=True, inplace=True)
copy

,Data,ABEV3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BEEF3,BPAC11,...,TAEE11,TIMP3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3,YDUQ3
0,12/06/2000,7.7383,NaN,NaN,2.0933,2.6461,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7499,26.6508,NaN,NaN,NaN
1,13/06/2000,7.7476,NaN,NaN,2.1000,2.4937,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7400,26.2794,NaN,NaN,NaN
2,14/06/2000,7.7414,NaN,NaN,2.1467,2.5022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7306,26.8365,NaN,NaN,NaN
3,15/06/2000,7.7552,NaN,NaN,2.1200,2.5243,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7256,26.4651,NaN,NaN,NaN
4,16/06/2000,7.7185,NaN,NaN,2.0367,2.6572,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7500,26.0936,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964,05/06/2020,14.0500,21.16,49.07,35.2000,20.4500,22.30,28.49,13.01,60.24,...,28.93,14.60,20.76,18.68,6.90,54.6100,47.7700,14.85,44.33,38.56
4965,08/06/2020,14.6000,27.35,51.72,36.3900,21.4300,23.47,28.40,13.05,66.02,...,28.94,14.44,21.03,19.13,7.45,54.7800,48.5400,15.50,45.80,37.15
4966,09/06/2020,14.3200,25.78,51.07,35.7900,21.2100,22.95,27.62,12.59,65.58,...,28.81,14.48,21.11,19.27,7.30,54.9900,49.0000,14.97,45.81,36.64
4967,10/06/2020,13.9600,23.50,50.74,34.2100,20.3600,21.88,26.85,12.77,63.36,...,29.05,14.30,21.49,18.62,6.89,54.2000,49.5900,14.95,44.99,34.90


In [128]:
copy = alpha.iloc[::1].copy()
copy.reset_index(drop=True, inplace=True)
copy

,Data,ABEV3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BEEF3,BPAC11,...,TAEE11,TIMP3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3,YDUQ3
0,12/06/2020,13.2200,21.22,48.87,32.9200,19.5600,21.40,25.97,12.71,61.34,...,28.52,13.88,20.87,17.70,6.53,52.4900,48.4100,14.76,43.62,33.17
1,10/06/2020,13.9600,23.50,50.74,34.2100,20.3600,21.88,26.85,12.77,63.36,...,29.05,14.30,21.49,18.62,6.89,54.2000,49.5900,14.95,44.99,34.90
2,09/06/2020,14.3200,25.78,51.07,35.7900,21.2100,22.95,27.62,12.59,65.58,...,28.81,14.48,21.11,19.27,7.30,54.9900,49.0000,14.97,45.81,36.64
3,08/06/2020,14.6000,27.35,51.72,36.3900,21.4300,23.47,28.40,13.05,66.02,...,28.94,14.44,21.03,19.13,7.45,54.7800,48.5400,15.50,45.80,37.15
4,05/06/2020,14.0500,21.16,49.07,35.2000,20.4500,22.30,28.49,13.01,60.24,...,28.93,14.60,20.76,18.68,6.90,54.6100,47.7700,14.85,44.33,38.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964,16/06/2000,7.7185,NaN,NaN,2.0367,2.6572,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7500,26.0936,NaN,NaN,NaN
4965,15/06/2000,7.7552,NaN,NaN,2.1200,2.5243,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7256,26.4651,NaN,NaN,NaN
4966,14/06/2000,7.7414,NaN,NaN,2.1467,2.5022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7306,26.8365,NaN,NaN,NaN
4967,13/06/2000,7.7476,NaN,NaN,2.1000,2.4937,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7400,26.2794,NaN,NaN,NaN


In [130]:
copy = copy.iloc[0:10].copy()
copy.reset_index(drop=True, inplace=True)
copy

,Data,ABEV3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BEEF3,BPAC11,...,TAEE11,TIMP3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3,YDUQ3
0,12/06/2020,13.22,21.22,48.87,32.92,19.56,21.40,25.97,12.71,61.34,...,28.52,13.88,20.87,17.70,6.53,52.49,48.41,14.76,43.62,33.17
1,10/06/2020,13.96,23.50,50.74,34.21,20.36,21.88,26.85,12.77,63.36,...,29.05,14.30,21.49,18.62,6.89,54.20,49.59,14.95,44.99,34.90
2,09/06/2020,14.32,25.78,51.07,35.79,21.21,22.95,27.62,12.59,65.58,...,28.81,14.48,21.11,19.27,7.30,54.99,49.00,14.97,45.81,36.64
3,08/06/2020,14.60,27.35,51.72,36.39,21.43,23.47,28.40,13.05,66.02,...,28.94,14.44,21.03,19.13,7.45,54.78,48.54,15.50,45.80,37.15
4,05/06/2020,14.05,21.16,49.07,35.20,20.45,22.30,28.49,13.01,60.24,...,28.93,14.60,20.76,18.68,6.90,54.61,47.77,14.85,44.33,38.56
5,04/06/2020,13.38,19.08,49.58,34.94,19.99,21.90,28.05,13.36,60.25,...,28.26,14.13,21.69,18.63,6.88,55.66,47.30,13.95,43.80,35.11
6,03/06/2020,13.25,18.45,48.82,34.75,19.91,21.59,27.89,12.45,59.50,...,28.71,13.67,21.48,18.76,6.90,53.66,47.25,13.48,42.60,35.14
7,02/06/2020,12.97,16.75,46.46,32.82,19.05,20.67,26.51,12.88,55.31,...,29.00,13.77,20.75,18.94,6.92,53.42,48.04,13.28,42.42,33.55
8,01/06/2020,12.60,15.35,45.53,31.80,18.37,19.80,25.65,13.08,50.14,...,28.97,13.51,20.44,17.19,6.19,53.42,47.40,13.43,42.16,30.03
9,29/05/2020,12.48,14.28,45.55,30.84,17.80,18.95,25.28,13.39,48.84,...,28.75,13.62,20.11,17.12,6.15,53.00,47.14,12.40,41.83,28.48


In [131]:
copy = copy.iloc[::-1].copy()
copy.reset_index(drop=True, inplace=True)
copy

,Data,ABEV3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BEEF3,BPAC11,...,TAEE11,TIMP3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3,YDUQ3
0,29/05/2020,12.48,14.28,45.55,30.84,17.80,18.95,25.28,13.39,48.84,...,28.75,13.62,20.11,17.12,6.15,53.00,47.14,12.40,41.83,28.48
1,01/06/2020,12.60,15.35,45.53,31.80,18.37,19.80,25.65,13.08,50.14,...,28.97,13.51,20.44,17.19,6.19,53.42,47.40,13.43,42.16,30.03
2,02/06/2020,12.97,16.75,46.46,32.82,19.05,20.67,26.51,12.88,55.31,...,29.00,13.77,20.75,18.94,6.92,53.42,48.04,13.28,42.42,33.55
3,03/06/2020,13.25,18.45,48.82,34.75,19.91,21.59,27.89,12.45,59.50,...,28.71,13.67,21.48,18.76,6.90,53.66,47.25,13.48,42.60,35.14
4,04/06/2020,13.38,19.08,49.58,34.94,19.99,21.90,28.05,13.36,60.25,...,28.26,14.13,21.69,18.63,6.88,55.66,47.30,13.95,43.80,35.11
5,05/06/2020,14.05,21.16,49.07,35.20,20.45,22.30,28.49,13.01,60.24,...,28.93,14.60,20.76,18.68,6.90,54.61,47.77,14.85,44.33,38.56
6,08/06/2020,14.60,27.35,51.72,36.39,21.43,23.47,28.40,13.05,66.02,...,28.94,14.44,21.03,19.13,7.45,54.78,48.54,15.50,45.80,37.15
7,09/06/2020,14.32,25.78,51.07,35.79,21.21,22.95,27.62,12.59,65.58,...,28.81,14.48,21.11,19.27,7.30,54.99,49.00,14.97,45.81,36.64
8,10/06/2020,13.96,23.50,50.74,34.21,20.36,21.88,26.85,12.77,63.36,...,29.05,14.30,21.49,18.62,6.89,54.20,49.59,14.95,44.99,34.90
9,12/06/2020,13.22,21.22,48.87,32.92,19.56,21.40,25.97,12.71,61.34,...,28.52,13.88,20.87,17.70,6.53,52.49,48.41,14.76,43.62,33.17


In [97]:
copy.reset_index(drop=True, inplace=True)
copy

,Data,ABEV3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BEEF3,BPAC11,...,TAEE11,TIMP3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3,YDUQ3
0,29/05/2020,12.48,14.28,45.55,30.84,17.80,18.95,25.28,13.39,48.84,...,28.75,13.62,20.11,17.12,6.15,53.00,47.14,12.40,41.83,28.48
1,01/06/2020,12.60,15.35,45.53,31.80,18.37,19.80,25.65,13.08,50.14,...,28.97,13.51,20.44,17.19,6.19,53.42,47.40,13.43,42.16,30.03
2,02/06/2020,12.97,16.75,46.46,32.82,19.05,20.67,26.51,12.88,55.31,...,29.00,13.77,20.75,18.94,6.92,53.42,48.04,13.28,42.42,33.55
3,03/06/2020,13.25,18.45,48.82,34.75,19.91,21.59,27.89,12.45,59.50,...,28.71,13.67,21.48,18.76,6.90,53.66,47.25,13.48,42.60,35.14
4,04/06/2020,13.38,19.08,49.58,34.94,19.99,21.90,28.05,13.36,60.25,...,28.26,14.13,21.69,18.63,6.88,55.66,47.30,13.95,43.80,35.11
5,05/06/2020,14.05,21.16,49.07,35.20,20.45,22.30,28.49,13.01,60.24,...,28.93,14.60,20.76,18.68,6.90,54.61,47.77,14.85,44.33,38.56
6,08/06/2020,14.60,27.35,51.72,36.39,21.43,23.47,28.40,13.05,66.02,...,28.94,14.44,21.03,19.13,7.45,54.78,48.54,15.50,45.80,37.15
7,09/06/2020,14.32,25.78,51.07,35.79,21.21,22.95,27.62,12.59,65.58,...,28.81,14.48,21.11,19.27,7.30,54.99,49.00,14.97,45.81,36.64
8,10/06/2020,13.96,23.50,50.74,34.21,20.36,21.88,26.85,12.77,63.36,...,29.05,14.30,21.49,18.62,6.89,54.20,49.59,14.95,44.99,34.90
9,12/06/2020,13.22,21.22,48.87,32.92,19.56,21.40,25.97,12.71,61.34,...,28.52,13.88,20.87,17.70,6.53,52.49,48.41,14.76,43.62,33.17


In [ ]:
alpha.iloc[::-1][0:50]

In [60]:
alpha

,Data,ABEV3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BEEF3,BPAC11,...,TAEE11,TIMP3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3,YDUQ3
0,12/06/2020,13.2200,21.22,48.87,32.9200,19.5600,21.40,25.97,12.71,61.34,...,28.52,13.88,20.87,17.70,6.53,52.4900,48.4100,14.76,43.62,33.17
1,10/06/2020,13.9600,23.50,50.74,34.2100,20.3600,21.88,26.85,12.77,63.36,...,29.05,14.30,21.49,18.62,6.89,54.2000,49.5900,14.95,44.99,34.90
2,09/06/2020,14.3200,25.78,51.07,35.7900,21.2100,22.95,27.62,12.59,65.58,...,28.81,14.48,21.11,19.27,7.30,54.9900,49.0000,14.97,45.81,36.64
3,08/06/2020,14.6000,27.35,51.72,36.3900,21.4300,23.47,28.40,13.05,66.02,...,28.94,14.44,21.03,19.13,7.45,54.7800,48.5400,15.50,45.80,37.15
4,05/06/2020,14.0500,21.16,49.07,35.2000,20.4500,22.30,28.49,13.01,60.24,...,28.93,14.60,20.76,18.68,6.90,54.6100,47.7700,14.85,44.33,38.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964,16/06/2000,7.7185,NaN,NaN,2.0367,2.6572,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7500,26.0936,NaN,NaN,NaN
4965,15/06/2000,7.7552,NaN,NaN,2.1200,2.5243,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7256,26.4651,NaN,NaN,NaN
4966,14/06/2000,7.7414,NaN,NaN,2.1467,2.5022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7306,26.8365,NaN,NaN,NaN
4967,13/06/2000,7.7476,NaN,NaN,2.1000,2.4937,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7400,26.2794,NaN,NaN,NaN


In [70]:
#Create a DataFrame
df1 = {
    'State':['Arizona AZ','Georgia GG','Newyork NY','Indiana IN','Florida FL'],
   'Score':[62,47,55,74,31]}
 
df1 = pd.DataFrame(df1,columns=['State','Score'])
df1

,State,Score
0,Arizona AZ,62
1,Georgia GG,47
2,Newyork NY,55
3,Indiana IN,74
4,Florida FL,31


In [69]:
df1.iloc[::-1]

,State,Score
4,Florida FL,31
3,Indiana IN,74
2,Newyork NY,55
1,Georgia GG,47
0,Arizona AZ,62
